In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


Устанавливаем нужны либы для обработки текста

In [2]:
%pip install re
%pip install nltk
%pip install unicodedata
%pip install contractions
%pip install inflect
%pip install emoji
%pip install pymorphy2

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
ERROR: Could not find a version that satisfies the requirement unicodedata (from versions: none)
ERROR: No matching distribution found for unicodedata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 68.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=8eb4d55a52e7145a43597e0c04768c7a20d427afe8c8221c31884e9e98acaed1
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successf

Импортируем и загружаем словари для удалений

In [3]:
import re
import nltk
import emoji
import unicodedata
import contractions
import inflect
import pymorphy2
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Часто в отзывах пользователи оставляют эмоджи, которые указывают эмоцииальную окраску, поэтому чтобы не терять такую информацию, преобразуем эмоции в текст

In [4]:
def emojis_words(text):

    # Модуль emoji: преобразование эмоджи в их словесные описания
    processed_text = emoji.demojize(text, delimiters=(" ", " "), language='ru')

    return processed_text

In [5]:
text = 'Безумно понравилось общение 😊 Я рада что в Тинькофф банке'
print(emojis_words(text))

Безумно понравилось общение  довольно_улыбается  Я рада что в Тинькофф банке


Избавимся от лишних символов, которые появляются при деэмоджировании)

In [6]:
def emojis_words(text):

    # Модуль emoji: преобразование эмоджи в их словесные описания
    processed_text = emoji.demojize(text, delimiters=(" ", " "), language='ru')

    # заменяет символы : и _ на пробел
    processed_text = processed_text.replace(":", "").replace("_", " ")

    return processed_text

In [7]:
text = 'Безумно понравилось общение 😊 Я рада что в Тинькофф банке'
print(emojis_words(text))

Безумно понравилось общение  довольно улыбается  Я рада что в Тинькофф банке


Далее напишем функцию для избавления от "всякого мусора" в отзывах при помощи регулярных выражений

In [8]:
def clean_review(review):
  # Приводим все к нижнему регистру
  cleaned_review = review.lower()

  # удаление HTML тегов
  cleaned_review = re.sub('<[^<]+?>', '', cleaned_review)

  # удаление цифр
  cleaned_review = ''.join([x for x in cleaned_review if not x.isdigit()])

  # удаление ссылок
  cleaned_review = re.sub(r'http\S+', '', cleaned_review)

  # Удаление стоп-слов - это стандартная практика очистки текстов
  stop_words = set(stopwords.words('russian'))
  tokens = word_tokenize(cleaned_review)
  tokens = [token for token in tokens if token not in stop_words]
  cleaned_review = ' '.join(tokens)

  # Удаляем из текста все знаки препинания
  cleaned_review = re.sub(r'[^\w\s]', '', cleaned_review)

  return cleaned_review


In [9]:
review = 'Очень оперативно и с результатом решили мои вопросы в чат поддержке: недавно у меня сняли деньги за непонятную подписку, банк решил вопрос за пару дней, деньги вернули. Также есть возможности и услуги, которые выручают в трудную минуту: кубышка, кредитная карта. Проценты небольгие, условия лучше, чем у контор с микрозаймами. В приложении всё просто, легко ориентироваться, есть возможность заказать доставку продуктов, записаться в салон красоты🙂. Спасибо'

In [10]:
clean_review(review)

'очень оперативно результатом решили мои вопросы чат поддержке  недавно сняли деньги непонятную подписку  банк решил вопрос пару дней  деньги вернули  также возможности услуги  которые выручают трудную минуту  кубышка  кредитная карта  проценты небольгие  условия  контор микрозаймами  приложении всё просто  легко ориентироваться  возможность заказать доставку продуктов  записаться салон красоты  спасибо'

Очищаем отзыв и лемматизируем

In [11]:
morph = pymorphy2.MorphAnalyzer()

def data_preprocessing(review):
  review = emojis_words(review)
  review = clean_review(review)

  # tokenization
  tokens = word_tokenize(review)

  # stop words removal
  review = [morph.parse(word)[0].normal_form for word in tokens]

  review = ' '.join(review)

  return review


print(data_preprocessing(review))

очень оперативно результат решить мой вопрос чат поддержка недавно снять деньга непонятный подписка банк решить вопрос пара день деньга вернуть также возможность услуга который выручать трудный минута кубышка кредитный карта процент небольгий условие контора микрозаём приложение всё просто легко ориентироваться возможность заказать доставка продукт записаться салон красота слегка улыбаться спасибо


Применим предобработку к нашему датасету

In [13]:
import pandas as pd

df = pd.read_csv('reviews.csv')
df.head()

,Unnamed: 0,header,created_date,rating,message
0,0,Поддержка,01.03.2024 22:39,5,Хочу выразить огромную благодарность сотрудник...
1,1,Персональный Менеджер,26.02.2024 14:48,5,Добрый день.
2,2,Самый лучший банк,09.03.2024 10:39,5,"Очень редко пишу отзывы, но сейчас то самое вр..."
3,3,Обращение по отправке годовой декларации в нал...,05.03.2024 18:15,5,Общалась с Ф-вой Натальей несколько дней. Снач...
4,4,Спасибо большое сотрудникам Тинькофф поддержки...,26.02.2024 16:01,5,"26,02 обратились в Тиньков по спорному моменту..."


In [14]:
df["message"] = df["message"].astype(str)
df.dtypes

Unnamed: 0       int64
header          object
created_date    object
rating           int64
message         object
dtype: object

In [15]:
df['preprocessed_message'] = df['message'].apply(lambda review: data_preprocessing(review))
df['preprocessed_header'] = df['header'].apply(lambda review: data_preprocessing(review))

df.head()

,Unnamed: 0,header,created_date,rating,message,preprocessed_message,preprocessed_header
0,0,Поддержка,01.03.2024 22:39,5,Хочу выразить огромную благодарность сотрудник...,хотеть выразить огромный благодарность сотрудн...,поддержка
1,1,Персональный Менеджер,26.02.2024 14:48,5,Добрый день.,добрый день,персональный менеджер
2,2,Самый лучший банк,09.03.2024 10:39,5,"Очень редко пишу отзывы, но сейчас то самое вр...",очень редко писать отзыв самый время самый хор...,самый хороший банк
3,3,Обращение по отправке годовой декларации в нал...,05.03.2024 18:15,5,Общалась с Ф-вой Натальей несколько дней. Снач...,общаться фвой наталья несколько день сначала у...,обращение отправка годовой декларация налоговый
4,4,Спасибо большое сотрудникам Тинькофф поддержки...,26.02.2024 16:01,5,"26,02 обратились в Тиньков по спорному моменту...",обратиться тиньковы спорный момент перевод ден...,спасибо большой сотрудник тинькофф поддержка б...


In [16]:
df["preprocessed_message"][1337]

'обратиться чат мобильный приложение вопрос оспаривание операция списание деньга карта вопрос решиться очень быстро деньга вернуть заниматься мария большой спасибо'

In [17]:
reviews = df["preprocessed_message"].values

reviews

array(['хотеть выразить огромный благодарность сотрудник тинькофф банк мва элвин чёткий быстрый решение поставить задача приятный общение считать такой работник гордиться брать пример работа культура общение клиент ваш отделение большой такой сотрудник хороший пример честность добропорядочность что надеяться другой реальный случай убедиться сказать большой спасибо это',
       'добрый день',
       'очень редко писать отзыв самый время самый хороший банк такой проблема решить оперативно быстро любой вопрос последний капля стать вернуть деньга мошеннический сайт думать увидеть деньга ребята огромный спасибо выбирать советовать весь правда очень крутой',
       ...,
       'мой банковский карта списать блатёж услуга сторонний организация мой согласить точнее год оплачивать услуга премиум телеграмма истечение срок действие оплатить услуга карта списаться стоимость год согласие давать автоплатёж подключать мой обращение менее неделя проблема банк решить деньга возвратить',
       'добрый д

векторизуем с помощью "мешка слов", min_df = 3\, чтобы убрать аномальные токены

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(min_df = 3, ngram_range=(1,2))

bow = count_vectorizer.fit_transform(reviews)
bow.shape

(4117, 11486)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10, learning_method="batch", max_iter=25, random_state=0)

topics = lda.fit_transform(bow)


In [75]:
lda.components_.shape

(10, 11486)

In [76]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(count_vectorizer.get_feature_names_out())

In [57]:
!pip install mglearn

In [78]:
mglearn.tools.print_topics(topics=range(10), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=10)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
кэшбэк        банк          банк          деньга        оператор      
покупка       счёт          вопрос        карта         поддержка     
начислить     тинькофф      очень         вернуть       курс          
категория     перевод       тинькофф      день          который       
кэшбек        комиссия      решить        банк          это           
повышенный    деньга        проблема      подписка      приложение    
вопрос        банкомат      быстро        поддержка     бек           
чат           день          поддержка     средство      кеш           
начисление    средство      спасибо       тинькофф      рубль         
кешбек        поддержка     хороший       списать       валюта        


topic 5       topic 6       topic 7       topic 8       topic 9       
--------      --------      --------      --------      --------      
банк

векторизуем с помощью TF-IDF, min_df = 3\, чтобы убрать аномальные токены

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer

v = TfidfVectorizer(min_df = 2, ngram_range=(1,2)).fit(reviews)

In [72]:
array_r = v.transform(reviews)

In [73]:
max_value = array_r.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()

f_names = np.array(v.get_feature_names_out())

In [74]:
print('Признаки с наименьшими td-idf:\n{}'.format(f_names[sorted_by_tfidf[:30]]))

print('Признаки с наибольшими td-idf:\n{}'.format(f_names[sorted_by_tfidf[-30:]]))



Признаки с наименьшими td-idf:
['оферта должный' 'инициатива иной' 'должный признаваться' 'анкета банк'
 'установить договор' 'ухудшить' 'акцептант который' 'документ выразить'
 'акцептант правопорядок' 'акцептант такой' 'договор условие'
 'отписка скрипт' 'договор оферент' 'договор норма' 'такой бремя'
 'иной грубо' 'правопорядок допускать' 'реагирование самый'
 'признаваться ничтожный' 'интерес адресат' 'ухудшить положение' 'ст гк'
 'исключительно собственный' 'кодекс российский'
 'обращение регистрировать' 'неудобный положение' 'давать глупый'
 'грубо посягать' 'положение вынудить' 'положение гражданский']
Признаки с наибольшими td-idf:
['заблокировать карта' 'описание ситуация' 'клуб' 'читать'
 'уважаемый тинькофф' 'обратиться поддержка' 'банк высота' 'опыт тинькофф'
 'премиум клиент' 'обращение продукт' 'отличный карта' 'день добрый'
 'заказывать' 'моментальный' 'душа' 'ноябрь' 'банк' 'здравствуйте'
 'февраль' 'иван' 'добрый' 'dmitry' 'год' 'отзыв' 'претензия'
 'приветствовать' 'о